<a href="https://colab.research.google.com/github/anilbhatt1/EVA4P1_S15_MaskRCNN/blob/master/EVA4P1_S15_DataPrep.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [32]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [0]:
from datetime import datetime 
print("Current Date/Time: ", datetime.now())

Current Date/Time:  2020-05-04 16:48:47.301193


In [0]:
#drive.flush_and_unmount()

In [0]:
import os
import PIL
from PIL import Image, ImageOps
from random import seed
from random import randint

In [0]:
def pos_generator(num1,num2,positions):
    lst = []
    for i in range(positions):
        x = randint(num1,num2)
        y = randint(num1,num2)
        tup = (x,y) 
        lst.append(tup)
    return lst    

In [0]:
path_bg    = '/content/gdrive/My Drive/EVA4P1_S15/BG_Images/'
path_fg    = '/content/gdrive/My Drive/EVA4P1_S15/FG_Images/'
path_fg_bg = '/content/sample_data/FG_BG/'
file1      = open('/content/sample_data/FG_BG_Filename_Logs.txt', "w")
num1       = 0
num2       = 100
positions  = 20
i          = 0

for bg_img in os.listdir(path_bg):
    if bg_img.endswith('jpg'):
       for fg_img in os.listdir(path_fg):
           if fg_img.endswith('png'):
              foreground = Image.open(f'{path_fg}{fg_img}') 
              pos1_list  = pos_generator(num1,num2, positions)
              pos2_list  = pos_generator(num1+10,num2+10, positions)
              for tup in range(len(pos1_list)):
                  i += 1
                  x,y = pos1_list[tup]
                  background = Image.open(f'{path_bg}{bg_img}')                            
                  background.paste(foreground, (x, y), foreground)
                  fg_bg_img_name  = 'Img_fg_bg_' + str(i) + '.jpg'
                  background.save(f'{path_fg_bg}{fg_bg_img_name}')
                  string = f'{path_fg_bg}{fg_bg_img_name}' + '\n'
                  file1.write(string)
                  i += 1
                  x,y = pos2_list[tup]
                  background_flip  = Image.open(f'{path_bg}{bg_img}')  
                  background_flip  = ImageOps.mirror(background_flip)
                  background_flip.paste(foreground, (x, y), foreground)                  
                  fg_bg_img_name = 'Img_fg_bg_' + str(i) + '.jpg'
                  background_flip.save(f'{path_fg_bg}{fg_bg_img_name}') 
                  string = f'{path_fg_bg}{fg_bg_img_name}' + '\n'
                  file1.write(string)                      
    if i % 100000 == 0:
       print('Progress: ',i,' fg_bg images saved so far ',datetime.now())
       print('fg_bg_img_name:',fg_bg_img_name)
print('Total Images saved:',i)
file1.close()

Progress:  100000  fg_bg images saved so far  2020-05-04 15:32:03.961139
fg_bg_img_name: Img_fg_bg_100000.jpg
Progress:  200000  fg_bg images saved so far  2020-05-04 15:40:37.839363
fg_bg_img_name: Img_fg_bg_200000.jpg
Progress:  300000  fg_bg images saved so far  2020-05-04 15:50:29.914947
fg_bg_img_name: Img_fg_bg_300000.jpg
Progress:  400000  fg_bg images saved so far  2020-05-04 15:59:48.223838
fg_bg_img_name: Img_fg_bg_400000.jpg
Total Images saved: 400000


In [0]:
file2 = open('/content/sample_data/SampleDrive_Filenames.txt', "w")
k = 0
path_fg_bg = '/content/sample_data/FG_BG/'
for fg_bg in os.listdir(path_fg_bg):
    string  = f'{path_fg_bg}{fg_bg}' + '\n'
    file2.write(string)
    k +=1
file2.close()  
print('No: of files in sample drive:',k)

No: of files in sample drive: 400000


In [0]:
!pip install -U -q PyDrive

from google.colab import files
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
import zipfile
import os
import sys

In [0]:
zipname = 'FG_BG'

def zipfolder(foldername, target_dir):            
    zipobj = zipfile.ZipFile(foldername + '.zip', 'w', zipfile.ZIP_DEFLATED)
    rootlen = len(target_dir) + 1
    for base, dirs, files in os.walk(target_dir):
        for file in files:
            fn = os.path.join(base, file)
            zipobj.write(fn, fn[rootlen:])

In [0]:
zipfolder(zipname, '/content/sample_data/FG_BG')

In [0]:
!cp FG_BG.zip '/content/gdrive/My Drive/EVA4P1_S15/'

In [0]:
drive.flush_and_unmount()

In [0]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [0]:
!unzip -q "/content/gdrive/My Drive/EVA4P1_S15/FG_BG.zip"

In [0]:
#le2 = open('/content/sample_data/SampleDrive_Filenames.txt', "w")
k = 0
path_fg_bg = '/content/sample_data/'
for file_name in os.listdir(path_fg_bg):
    k +=1
print('No: of files in sample drive:',k)